In [1]:
import sys, platform
import ctypes, ctypes.util
import shutil

ctypes.util.find_library("libSystem.B.dylib")
print(shutil.which("libSystem.B.dylib"))
ctypes.CDLL("libSystem.B.dylib")._name

None


'libSystem.B.dylib'

In [2]:
import os
# print(os.environ['DYLD_LIBRARY_PATH'])
os.environ['DYLD_LIBRARY_PATH'] = "/usr/local/opt/gcc/lib/gcc/10"
print(os.environ['DYLD_LIBRARY_PATH'])

/usr/local/opt/gcc/lib/gcc/10


In [4]:
%tb

import sys, platform
import ctypes, ctypes.util

# https://solarianprogrammer.com/2019/07/18/python-using-c-cpp-libraries-ctypes/
mylib_path = ctypes.util.find_library("full_cpu")
if not mylib_path:
    print("Unable to find the specified library.")
    sys.exit()

try:
    jesture_lib = ctypes.CDLL(mylib_path)
except OSError:
    print("Unable to load the system C library")
    sys.exit()

SystemExit: 

Unable to load the system C library


SystemExit: 

In [ ]:
create_full_cpu = jesture_lib.CreateFullCpu
create_full_cpu.argtypes = None
create_full_cpu.restype = ctypes.POINTER(ctypes.c_int)

run_full_cpu = jesture_lib.RunFullCpu
run_full_cpu.argtypes = [ctypes.POINTER(ctypes.c_int)]
run_full_cpu.restype = None

test_dispose = jesture_lib.DisposeFullCpu
test_dispose.argtypes = [ctypes.POINTER(ctypes.c_int)]
test_dispose.restype = None

get_dynamic_gesture = jesture_lib.GetCurrentDynamicGesture
get_dynamic_gesture.argtypes = [ctypes.POINTER(ctypes.c_int)]
get_dynamic_gesture.restype = ctypes.c_char_p

get_static_left_gesture = jesture_lib.GetCurrentStaticLeftGesture
get_static_left_gesture.argtypes = [ctypes.POINTER(ctypes.c_int)]
get_static_left_gesture.restype = ctypes.c_char_p

get_static_right_gesture = jesture_lib.GetCurrentStaticRightGesture
get_static_right_gesture.argtypes = [ctypes.POINTER(ctypes.c_int)]
get_static_right_gesture.restype = ctypes.c_char_p

# get_hand_left_keypoints = jesture_lib.GetCurrentHandLeftKeypoints
# get_hand_left_keypoints.argtypes = [ctypes.POINTER(ctypes.c_int)]
# get_hand_left_keypoints.restype = [ctypes.POINTER(ctypes.c_float)]

# get_hand_right_keypoints = jesture_lib.GetCurrentHandRightKeypoints
# get_hand_right_keypoints.argtypes = [ctypes.POINTER(ctypes.c_int)]
# get_hand_right_keypoints.restype = [ctypes.POINTER(ctypes.c_float)]